In [37]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from pyomo.contrib.solver.solvers import gurobi_direct
from pyomo.contrib.solver.solvers import ipopt
from pyomo.environ import *

import pandas as pd
import time

In [36]:
# Defining the model
model = pyo.ConcreteModel()
# Define index set
model.I = pyo.RangeSet(1,5)
# Model variables
model.x = pyo.Var(model.I, bounds = (0, None), within = Integers)
model.y = pyo.Var(bounds = (0, None))
x = model.x
y = model.y

# Model constraints
x_sum = sum(x[i] for i in model.I) # variables
model.c1 = pyo.Constraint(expr = x_sum + y <= 20)
model.c2 = pyo.ConstraintList()
for i in model.I:
    model.c2.add(expr = x[i]+y >= 15)

model.c3 = pyo.Constraint(expr = sum(i*x[i] for i in model.I) >=10)
model.c4 = pyo.Constraint(expr = x[5] + 2*y >= 30)

# Model objective function
model.obj = pyo.Objective(expr = x_sum + y, sense = minimize)


In [39]:
# Solve using Gurobi direct
begin_time = time.time()
solver = SolverFactory('gurobi_direct')
results = solver.solve(model, tee=True)
delta_t = time.time()-begin_time
print(delta_t)

Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8 rows, 6 columns and 23 nonzeros
Model fingerprint: 0x35883aef
Variable types: 1 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+01]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 7 rows, 6 columns, 21 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)
Found heuristic solution: objective 18.0000000

Root relaxation: objective 1.700000e+01, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    

In [40]:
model.pprint()

1 RangeSet Declarations
    I : Dimen=1, Size=5, Bounds=(1, 5)
        Key  : Finite : Members
        None :   True :   [1:5]

2 Var Declarations
    x : Size=5, Index=I
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :  None : False : False : Integers
          2 :     0 :  -0.0 :  None : False : False : Integers
          3 :     0 :  -0.0 :  None : False : False : Integers
          4 :     0 :  -0.0 :  None : False : False : Integers
          5 :     0 :   2.0 :  None : False : False : Integers
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  15.0 :  None : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[1] + x[2] + x[3] + x[4] + x[5] + y

4 Constraint Declarations
    c1 : Size=1, Index=None, Active=True
        Key  : Lower : Body                     

In [42]:
pyo.value(y), [pyo.value(x[i]) for i in model.I], pyo.value(model.obj)

(15.0, [0.0, -0.0, -0.0, -0.0, 2.0], 17.0)